# Test notebook for visualizing in 3D, Neurohackweek 2017

- Cloned from the demo introductory_3d.ipynb notebook
- Initial functionality based on [IPyvolume](https://ipyvolume.readthedocs.io/en/latest/index.html)
- modified by melanie to focus on surface viewer

load packages

In [70]:
%matplotlib inline
import matplotlib.pyplot as plt

import nibabel as nb
import ipyvolume
import ipyvolume.pylab as p3
import numpy as np
import os
import sys

In [71]:
# before running, set the global variable NHW to your NHW_visualization git directory
# e.g. add 'export NHW=/Users/ely/Desktop/nhw2017/NHW_visualization' to your ~/.bashrc file
nhw = os.environ['NHW'] # set the global variable NHW to your NHW_visualization git directory
V = nb.load(nhw + '/data/100610/MNINonLinear/T1w_restore_brain.nii.gz').get_data()

In [72]:
def showSurface(surface,overlay=None,figure=None,colormap='autumn',figsize=np.array([600,600]),
                figlims=np.array([[-100,100],[-100,100],[-100,100]])):
    
    if isinstance(surface,str):
        if not os.path.exists(surface):
            error('File does not exist, please provide a valid file path to a gifti or FreeSurfer file.')
        filename, file_extension = os.path.splitext(surface)
        if file_extension=='.gii':
            surface = nb.load(surface)
        else:
            fsgeometry = nb.freesurfer.read_geometry(surface)
            x,y,z = fsgeometry[0].T
            vertex_edges=fsgeometry[1]

    if isinstance(surface,nb.gifti.gifti.GiftiImage):
        try:
            vertex_spatial=surface.darrays[0]
            vertex_edges=surface.darrays[1]
            x, y, z = vertex_spatial.data.T
        except:
            raise ValueError('Please provide a valid gifti file.')
        
    if isinstance(overlay,str):
        if not os.path.exists(surface):
            error('File does not exist, please provide a valid file path to a gifti or FreeSurfer file.')
        filename, file_extension = os.path.splitext(overlay)
        
        if file_extension=='.gii':
            overlay = nb.load(overlay)
        elif file_extension=='.annot':
            annot = nb.freesurfer.read_annot(overlay)
            activation = annot[0]
        elif file_extension=='.curv':
            activation = nb.freesurfer.read_morph_data(overlay)
            
    if isinstance(overlay,nb.gifti.gifti.GiftiImage):
        try:
            activation=overlay.darrays[0].data
        except:
            raise ValueError('Please provide a valid gifti file')
    
    
    if figure is None:

        fig = p3.figure(width=figsize[0], height=figsize[0])
        fig.camera_fov = 1
        fig.style = {'axes': {'color': 'black',
          'label': {'color': 'black'},
          'ticklabel': {'color': 'black'},
          'visible': False},
         'background-color': 'white',
         'box': {'visible': False}}
        fig.xlim = (figlims[0][0], figlims[0][1])
        fig.ylim = (figlims[1][0], figlims[1][1])
        fig.zlim = (figlims[2][0], figlims[2][1])

        
    # plot surface
    if overlay is None:
        p3.plot_trisurf(x, y, z, triangles=vertex_edges.data)
    else:
        my_color = plt.cm.get_cmap(colormap)
        colors=my_color((activation-min(activation))/(max(activation)-min(activation)))
        p3.plot_trisurf(x, y, z, triangles=vertex_edges.data, color=colors[:,:3])

    if figure is None:
        p3.show()
    
    
   

## Plot gifti surface and overlay files for a subject's left and right hemisphere

In [73]:
data_lh = nb.load(nhw+'/data/100610/MNINonLinear/fsaverage_LR32k/100610.L.midthickness_MSMAll.32k_fs_LR.surf.gii')
data_lh_myelin = nb.load(nhw+'/data/100610/MNINonLinear/fsaverage_LR32k/100610.L.MyelinMap.32k_fs_LR.func.gii')

data_rh = nb.load(nhw+'/data/100610/MNINonLinear/fsaverage_LR32k/100610.R.midthickness_MSMAll.32k_fs_LR.surf.gii')
data_rh_myelin = nb.load(nhw+'/data/100610/MNINonLinear/fsaverage_LR32k/100610.R.MyelinMap.32k_fs_LR.func.gii')

In [74]:
showSurface(data_lh,data_lh_myelin)

A Jupyter Widget

In [75]:
showSurface(data_rh,data_rh_myelin,1)

## now read in a FreeSurfer surface and curvature/annotation as overlay

In [76]:
showSurface(nhw+'/data/006/surf/lh.inflated',nhw+'/data/006/surf/lh.curv')

reads activation in


A Jupyter Widget

In [77]:
showSurface(nhw+'/data/006/surf/lh.inflated',nhw+'/data/006/surf/lh.aparc.annot')

A Jupyter Widget